# Описание проекта:

**Мотивация:** Среди моих знакомых и родных в последнее время часто идут разговоры о недвижимости в москве,  у кого-то она есть, а кто-то планирует переезжать да и я сам хочу пресмотреть что-нибудь в МО. Поэтому решил разобраться в этой теме и используя отрытые данные построить модель оценки стоимости домов и таунхаусов (позже возможно добавлю квартиры).

**Цель:** Создать модель машинного обучения для предсказания стоимости недвижимости (дома и таунхаусы) в Москве и подмосковье с использованием данных, полученных с помощью библиотеки Cianparser.

**Целевая аудитория:**

* Жители Москвы и московской области, владеющие недвижимостью и желающие оценить её стоимость.
* Люди, планирующие переезд в Раменское или Жуковский и интересующиеся ценами на недвижимость.

**Инструменты:**

* Python
* Библиотека Cianparser для сбора данных
* Библиотеки машинного обучения (Scikit-learn)
* Streamlit для создания интерактивного приложения

**Ожидаемые результаты:**

* Модель машинного обучения с MAPE <= 0.2, способная точно предсказывать стоимость недвижимости.
* Интерактивное приложение Streamlit, позволяющее пользователям вводить характеристики недвижимости и получать прогноз цены.

## Парсинг данных

In [1]:
import time
import chime
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import cianparser
import requests
import re

chime.theme('mario')
%load_ext chime

In [2]:
locs = pd.DataFrame(cianparser.list_locations()) 
locs.columns = ['Город','Индекс']

In [13]:
locs[locs['Город'] == 'Красногорск']

,Город,Индекс
126,Красногорск,175071


In [17]:
locations = ['Жуковский','Раменское','Одинцово','Домодедово',
            'Реутов','Железнодорожный','Долгопрудный','Москва',
            'Красногорск','Подольск','Балашиха']

In [5]:
def parce_suburban(city, suburban_types = ["house",'townhouse'], pages_counts = [[1,1]],parsed_pages=0):
    parser = cianparser.CianParser(location=city)

    data = []

    for suburban_type,pages_count in zip(suburban_types,pages_counts):
        for page in range(pages_count[0], pages_count[1]+1):
            if parsed_pages >= 2:
                num = np.random.randint(110,120)
                print(f'parsed pages: {parsed_pages}')
                print(f'timeout: {num}')
                time.sleep(num)
                
            suburban_sale = parser.get_suburban( 
                                                deal_type="sale",
                                                suburban_type = suburban_type,
                                                with_extra_data=True,
                                                additional_settings={"start_page":page,
                                                                    "end_page":page}
                                                )
            data.append(pd.DataFrame(suburban_sale))
            parsed_pages +=1

    df = pd.concat(data).drop_duplicates().reset_index(drop=True)
    return df

In [6]:
%%time
%%chime
dfs = []

for parsed_pages,location in enumerate(locations):
    
    df = parce_suburban(location, pages_counts = [[1,1],[1,1]],
                        parsed_pages = parsed_pages*2)
    dfs.append(df)


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4750&deal_type=sale&offer_type=suburban&object_type%5B0%5D=1

 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %. Average price: 35 749 242 rub

The collection of information from the pages with list of offers is completed
Total number of parsed offers: 28. 

                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4750&deal_type=sale&offer_type=suburban&object_type%5B0%5D=4

 1 | 1 page with list: [=>=>=>=>=>=>] 100% | Count of all parsed: 6. Progress ratio: 100 %. Average price: 36 770 833 rub

The collection of information from the pages with list of o

In [15]:
locs[locs['Город']=='Балашиха']

,Город,Индекс
109,Балашиха,174292


In [16]:
%%time
%%chime

dfs = []

dfs.append(parce_suburban('Подольск', pages_counts = [[1,4]],suburban_types=['house'],
                            parsed_pages = 0))

dfs.append(parce_suburban('Красногорск', pages_counts = [[1,4],[1,2]],
                    parsed_pages = 4))

dfs.append(parce_suburban('Балашиха', pages_counts = [[1,8],[1,1]],
                    parsed_pages = 10))


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=4935&deal_type=sale&offer_type=suburban&object_type%5B0%5D=1

 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %. Average price: 49 221 489 rub

The collection of information from the pages with list of offers is completed
Total number of parsed offers: 28. 

                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=2&with_neighbors=0&region=4935&deal_type=sale&offer_type=suburban&object_type%5B0%5D=1

 1 | 2 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>                ] 72% | Count of all parsed: 20. Progress ratio: 72 %. Average price: 29 316 876 rub

Exception: ('Con

In [18]:
df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
df['date'] = dt.date.today()
df

,author,author_type,url,location,deal_type,accommodation_type,suburban_type,price_per_month,commissions,price,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,date
0,Монолит Столица,real_estate_agent,https://podolsk.cian.ru/sale/suburban/299274329/,Подольск,sale,suburban,house,-1,0,12500000,...,-1,-1,"85,1 м²",2,+79651405728,,м. Подольск,переулок Крымский,2/27,2024-04-24
1,ID 87392027,unknown,https://podolsk.cian.ru/sale/suburban/300560807/,Подольск,sale,suburban,house,-1,0,25000000,...,-1,-1,185 м²,2,+79851813193,,м. Подольск,улица Малая Северовская,20,2024-04-24
2,ID 53733,realtor,https://podolsk.cian.ru/sale/suburban/300899069/,Подольск,sale,suburban,house,-1,0,1900000,...,-1,-1,52 м²,2,+79683348903,,м. Подольск,улица Коммунальная,,2024-04-24
3,ЭТАЖИ Подольск,real_estate_agent,https://podolsk.cian.ru/sale/suburban/299302858/,Подольск,sale,suburban,house,-1,0,11000000,...,-1,-1,62 м²,2,+79684552495,,м. Силикатная,Силикатная улица,,2024-04-24
4,ЭТАЖИ Подольск,real_estate_agent,https://podolsk.cian.ru/sale/suburban/298492436/,Подольск,sale,suburban,house,-1,0,8500000,...,-1,-1,200 м²,3,+79645599647,,м. Подольск,улица Малая Зеленовская,,2024-04-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Home Market,real_estate_agent,https://balashikha.cian.ru/sale/suburban/29298...,Балашиха,sale,suburban,townhouse,-1,0,26500000,...,-1,-1,"229,6 м²",3,+79660505643,,,шоссе Леоновское,6к7,2024-04-24
426,Светлана Заремба,realtor,https://balashikha.cian.ru/sale/suburban/27844...,Балашиха,sale,suburban,townhouse,-1,0,27499000,...,-1,-1,260 м²,3,+79623689260,,,улица Студеная,,2024-04-24
427,Гульнара Абдуллина,realtor,https://balashikha.cian.ru/sale/suburban/28407...,Балашиха,sale,suburban,townhouse,-1,0,17350000,...,-1,-1,-1,2,+79167257220,,м. Салтыковская,Граничная улица,14,2024-04-24
428,АН Мегаполис-Сервис Юбилейная 8,real_estate_agent,https://balashikha.cian.ru/sale/suburban/30068...,Балашиха,sale,suburban,townhouse,-1,0,14990000,...,-1,-1,"85,5 м²",2,+79684988072,,м. Салтыковская,улица Рудневой,47,2024-04-24


In [19]:
old_df.shape

(2279, 27)

In [20]:
%%chime
old_df = pd.read_csv('suburban.csv')

new_df = df[[x not in old_df['url'].to_list() for x in df['url'].to_list()]]
new_size = new_df.shape[0]
display(new_df)

,author,author_type,url,location,deal_type,accommodation_type,suburban_type,price_per_month,commissions,price,...,sewage_system,bathroom,living_meters,floors_count,phone,district,underground,street,house_number,date
0,Монолит Столица,real_estate_agent,https://podolsk.cian.ru/sale/suburban/299274329/,Подольск,sale,suburban,house,-1,0,12500000,...,-1,-1,"85,1 м²",2,+79651405728,,м. Подольск,переулок Крымский,2/27,2024-04-24
1,ID 87392027,unknown,https://podolsk.cian.ru/sale/suburban/300560807/,Подольск,sale,suburban,house,-1,0,25000000,...,-1,-1,185 м²,2,+79851813193,,м. Подольск,улица Малая Северовская,20,2024-04-24
2,ID 53733,realtor,https://podolsk.cian.ru/sale/suburban/300899069/,Подольск,sale,suburban,house,-1,0,1900000,...,-1,-1,52 м²,2,+79683348903,,м. Подольск,улица Коммунальная,,2024-04-24
3,ЭТАЖИ Подольск,real_estate_agent,https://podolsk.cian.ru/sale/suburban/299302858/,Подольск,sale,suburban,house,-1,0,11000000,...,-1,-1,62 м²,2,+79684552495,,м. Силикатная,Силикатная улица,,2024-04-24
4,ЭТАЖИ Подольск,real_estate_agent,https://podolsk.cian.ru/sale/suburban/298492436/,Подольск,sale,suburban,house,-1,0,8500000,...,-1,-1,200 м²,3,+79645599647,,м. Подольск,улица Малая Зеленовская,,2024-04-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Home Market,real_estate_agent,https://balashikha.cian.ru/sale/suburban/29298...,Балашиха,sale,suburban,townhouse,-1,0,26500000,...,-1,-1,"229,6 м²",3,+79660505643,,,шоссе Леоновское,6к7,2024-04-24
426,Светлана Заремба,realtor,https://balashikha.cian.ru/sale/suburban/27844...,Балашиха,sale,suburban,townhouse,-1,0,27499000,...,-1,-1,260 м²,3,+79623689260,,,улица Студеная,,2024-04-24
427,Гульнара Абдуллина,realtor,https://balashikha.cian.ru/sale/suburban/28407...,Балашиха,sale,suburban,townhouse,-1,0,17350000,...,-1,-1,-1,2,+79167257220,,м. Салтыковская,Граничная улица,14,2024-04-24
428,АН Мегаполис-Сервис Юбилейная 8,real_estate_agent,https://balashikha.cian.ru/sale/suburban/30068...,Балашиха,sale,suburban,townhouse,-1,0,14990000,...,-1,-1,"85,5 м²",2,+79684988072,,м. Салтыковская,улица Рудневой,47,2024-04-24


In [21]:
if new_df.shape[0] > 0:
    full_df = (pd.concat([old_df,new_df])
            .drop_duplicates(subset=['url','author'],ignore_index=True)
            .reset_index(drop=True))

    full_df.to_csv('suburban.csv',index=False)

    
    print(f'Количество новых обьялений: {new_size}')

else:
    print('Новых обьявлений не обнаруженно')

print(f'Суммарное количество объялений: {full_df.shape[0]}')


Количество новых обьялений: 428
Суммарное количество объялений: 2812


In [93]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")  # user_agent нужен для идентификации приложения

# Получение координат центра Москвы
moscow_location = geolocator.geocode("Москва, Россия")
moscow_coords = (moscow_location.latitude, moscow_location.longitude)

# Получение координат центра Раменского
ramenskoye_location = geolocator.geocode("Раменское, Московская область, Россия")
ramenskoye_coords = (ramenskoye_location.latitude, ramenskoye_location.longitude)

# Получение координат центра Жуковского
zhukovsky_location = geolocator.geocode("Жуковский, Московская область, Россия")
zhukovsky_coords = (zhukovsky_location.latitude, zhukovsky_location.longitude)

# Пример геокодирования объекта недвижимости
address = "улица Ленина, 10, Раменское район, Московская область, Россия"
location = geolocator.geocode(address)
property_coords = (location.latitude, location.longitude)

In [94]:
location

Location(улица Ленина, Красный Октябрь, Раменское, Раменский городской округ, Московская область, Центральный федеральный округ, 140104, Россия, (55.5799671, 38.1908862, 0.0))

In [96]:
zhukovsky_coords

(55.5972801, 38.1199863)

In [95]:
property_coords

(55.5799671, 38.1908862)